# 类的描述符

## 是什么:
描述符本质就是一个新式类,在这个新式类中,至少实现了\_\_get\_\_(),\_\_set\_\_(),\_\_delete\_\_()中的一个,这也被称为描述符协议

\_\_get\_\_():调用一个属性时,触发

\_\_set\_\_():为一个属性赋值时,触发

\_\_delete\_\_():采用del删除属性时,触发

**\_\_get\_\_(),\_\_set\_\_(),\_\_delete\_\_()什么时候触发？**

In [1]:
class Foo(object):
    def __get__(self, instance, owner):
        print('触发get')
    def __set__(self, instance, value):
        print('触发set')
    def __delete__(self, instance):
        print('触发delete')
f = Foo()

In [2]:
f.name = 'alex'
print(f.name)
del f.name

alex


可以看到对类属性进行操作时候，并没有触发那三个方法。描述符没有用吗？　不是的，是打开的方式不对

## 干什么的
是用来代理另外一个类的属性的(必须把描述符定义成这个类的类属性，不能定义到构造函数中

In [3]:
class Str(object):
    def __get__(self, instance, owner):
        print('Str调用')

    def __set__(self, instance, value):
        print('Str设置...')

    def __delete__(self, instance):
        print('Str删除...')


class People(object):
    name = Str()

    def __init__(self, name):
        self.name = name


In [4]:
people = People('alxe')

Str设置...


In [5]:
people.name

Str调用


In [6]:
del people.name

Str删除...


在把描述符定义成这个类的类属性的时候，对这个类属性进行操作，就会调用

## 什么时候用

### 实现类型限制功能

In [7]:
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, owner):
        print('get--->', instance, owner)
        if instance is None:
            return self
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        print('set--->', instance, value)
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected %s' % str(self.expected_type))
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        print('delete--->', instance)
        instance.__dict__.pop(self.name)


class People:
    name = Typed('name', str)
    age = Typed('name', int)
    salary = Typed('name', float)

    def __init__(self, name, age, salary):
        self.name = name
        self.age = age
        self.salary = salary

In [8]:
p1=People(123,18,3333.3)

set---> <__main__.People object at 0x7fc4041ecef0> 123


TypeError: Expected <class 'str'>

In [9]:
p1=People('egon','18',3333.3)

set---> <__main__.People object at 0x7fc4041ec5f8> egon
set---> <__main__.People object at 0x7fc4041ec5f8> 18


TypeError: Expected <class 'int'>

In [10]:
p1=People('egon',18,3333)

set---> <__main__.People object at 0x7fc4041fb5c0> egon
set---> <__main__.People object at 0x7fc4041fb5c0> 18
set---> <__main__.People object at 0x7fc4041fb5c0> 3333


TypeError: Expected <class 'float'>

**实现了类型限制功能**

#### 装饰器版

In [11]:
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, owner):
        print('get--->', instance, owner)
        if instance is None:
            return self
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        print('set--->', instance, value)
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected %s' % str(self.expected_type))
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        print('delete--->', instance)
        instance.__dict__.pop(self.name)


def typeassert(**kwargs):
    def decorate(cls):
        print('类的装饰器开始运行啦------>', kwargs)
        for name, expected_type in kwargs.items():
            setattr(cls, name, Typed(name, expected_type))
        return cls

    return decorate


@typeassert(
    name=str, age=int, salary=float
)  #有参:1.运行typeassert(...)返回结果是decorate,此时参数都传给kwargs 2.People=decorate(People)
class People:
    def __init__(self, name, age, salary):
        self.name = name
        self.age = age
        self.salary = salary

p1 = People('egon', 18, 3333.3)

类的装饰器开始运行啦------> {'name': <class 'str'>, 'age': <class 'int'>, 'salary': <class 'float'>}
set---> <__main__.People object at 0x7fc404175a58> egon
set---> <__main__.People object at 0x7fc404175a58> 18
set---> <__main__.People object at 0x7fc404175a58> 3333.3


### 描述符实现静态方法staticmethod

In [13]:
class Staticmethod:
    def __init__(self, funcname):
        self.funcname = funcname

    def __get__(self, instance, cls):
        def wrappers(*kw, **kwargs):
            k = self.funcname(*kw, **kwargs)
            return k
        return wrappers

### 描述符实现classmethod

In [14]:
class Classmethod:  # 类装饰器写法
    def __init__(self, funcname):
        self.funcname = funcname

    def __get__(self, instance, cls):
        def wrappers(*kw, **kwargs):
            k = self.funcname(cls, *kw, **kwargs)
            return k

        return wrappers

### 描述符实现property

In [15]:
class Property:
    def __init__(self,func):
        self.func=func
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.func(instance) 

### property拓展
一个静态属性property本质就是实现了get，set，delete三种方法

In [16]:
class Foo:
    @property
    def AAA(self):
        print('get的时候运行我啊')

    @AAA.setter
    def AAA(self,value):
        print('set的时候运行我啊')

    @AAA.deleter
    def AAA(self):
        print('delete的时候运行我啊')

#只有在属性AAA定义property后才能定义AAA.setter,AAA.deleter

In [17]:
f1=Foo()
f1.AAA
f1.AAA='aaa'
del f1.AAA

get的时候运行我啊
set的时候运行我啊
delete的时候运行我啊


**如何用**

#### 1. 自动折扣功能

In [18]:
class Goods:

    def __init__(self):
        # 原价
        self.original_price = 100
        # 折扣
        self.discount = 0.8

    @property
    def price(self):
        # 实际价格 = 原价 * 折扣
        new_price = self.original_price * self.discount
        return new_price

    @price.setter
    def price(self, value):
        self.original_price = value

    @price.deleter
    def price(self):
        del self.original_price

In [19]:
obj = Goods()
obj.price         # 获取商品价格
obj.price = 200   # 修改商品原价
print(obj.price)
del obj.price     # 删除商品原价

160.0


#### 2. 类型限制功能

In [20]:
class People:
    def __init__(self,name):
        self.name=name #实例化就触发property

    @property
    def name(self):
        # return self.name #无限递归
        print('get------>')
        return self.DouNiWan

    @name.setter
    def name(self,value):
        print('set------>')
        if not isinstance(value,str):
            raise TypeError('必须是字符串类型')
        self.DouNiWan=value

    @name.deleter
    def name(self):
        print('delete------>')
        del self.DouNiWan

In [21]:
p1=People('alex') 
p1.name=1

set------>
set------>


TypeError: 必须是字符串类型

### 属性的查找顺序

优先级由高到底分别是
1. 类属性
2. 数据描述符
3. 实例属性
4. 非数据描述符
5. 找不到的属性触发__getattr__()